导入必要包
这里有个小提示，当aiohttp包因为vpn等原因无法正确处理session时候，可以使用httpx等不同包处理

In [6]:
import httpx
import asyncio

class SteamGameChecker:
    def __init__(self, desired_currency):
        self.desired_currency = desired_currency

    async def get_information(self, client, appid):
        url = f'http://store.steampowered.com/api/appdetails?appids={appid}&cc={self.desired_currency}&filters=price_overview'
        for _ in range(3):  # 尝试3次
            try:
                response = await client.get(url, timeout=20, follow_redirects=True)
                response.raise_for_status()
                j = response.json()
                data = j.get(str(appid), {}).get('data', None)
                if data and 'price_overview' in data:
                    price_info = data['price_overview']
                    return {
                        "initialprice": price_info['initial'] / 100,
                        "finalprice": price_info['final'] / 100,
                        "discountpercent": price_info['discount_percent'],
                        "currency": price_info['currency']
                    }
                else:
                    return {"error": "No pricing information available"}
            except httpx.HTTPStatusError as e:
                error_msg = str(e)
                await asyncio.sleep(2)  # 等待2秒后重试
        return {"error": error_msg}

async def fetch_all_appids(appids, desired_currency):
    async with httpx.AsyncClient() as client:
        checker = SteamGameChecker(desired_currency)
        tasks = [checker.get_information(client, appid) for appid in appids]
        responses = await asyncio.gather(*tasks)
        return responses

# Example usage
appids = [220200, 230410, 440]
desired_currency = 'USD'

# 在Notebook环境中直接调用异步函数
async def main():
    results = await fetch_all_appids(appids, desired_currency)
    for result in results:
        print(result)

await main()


{'initialprice': 39.0, 'finalprice': 39.0, 'discountpercent': 0, 'currency': 'SGD'}
{'error': 'No pricing information available'}
{'error': 'No pricing information available'}
